In [1]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForSequenceClassification
import os
model_id='Qwen/Qwen3-0.6B'
hf_token=
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    trust_remote_code=True,
    token=hf_token)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side='right'

In [2]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype='float16',
    bnb_4bit_use_double_quant=True
)

In [3]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=5,
    device_map='auto',
    quantization_config=bnb_config,
    token=hf_token  # 토큰 추가
)

model.config.use_cache = False
model.config.pad_token_id = tokenizer.pad_token_id

Some weights of Qwen3ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen3-0.6B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.preprocessing import LabelEncoder
import numpy as np

# -------------------------------
# 1. 엑셀 데이터 로드
# -------------------------------

# 엑셀 파일 경로 (수정 필요)
train_excel_path = "C:/Users/nice.DESKTOP-0JCR5PF/Desktop/train.xlsx"  # 200행
test_excel_path = "C:/Users/nice.DESKTOP-0JCR5PF/Desktop/test.xlsx"    # 50행

# 엑셀 읽기
train_df = pd.read_excel(train_excel_path)
test_df = pd.read_excel(test_excel_path)

print(f"Train 데이터 크기: {train_df.shape}")
print(f"Test 데이터 크기: {test_df.shape}")
print(f"칼럼: {list(train_df.columns)}")

# -------------------------------
# 2. 텍스트 칼럼들을 합치기
# -------------------------------

text_columns = ['발명의 명칭', '요약', '전체청구항', '대표청구항']

def combine_text_columns(row):
    """4개 텍스트 칼럼을 하나로 합치기"""
    texts = []
    for col in text_columns:
        if pd.notna(row[col]) and str(row[col]).strip():  # null이 아니고 빈 문자열이 아닌 경우
            texts.append(f"{col}: {str(row[col]).strip()}")
    return " | ".join(texts)

# 텍스트 칼럼 합치기
train_df['text'] = train_df.apply(combine_text_columns, axis=1)
test_df['text'] = test_df.apply(combine_text_columns, axis=1)

print(f"\n합쳐진 텍스트 예시:")
print(train_df['text'].iloc[0][:200] + "...")

# -------------------------------
# 3. 라벨 인코딩
# -------------------------------

# 사용자태그를 라벨로 변환
label_encoder = LabelEncoder()

# train + test 전체 라벨로 인코더 학습 (일관성 유지)
all_labels = pd.concat([train_df['사용자태그'], test_df['사용자태그']])
label_encoder.fit(all_labels.dropna())

# 라벨 인코딩 적용
train_df['label'] = label_encoder.transform(train_df['사용자태그'])
test_df['label'] = label_encoder.transform(test_df['사용자태그'])

# 라벨 정보 출력
unique_labels = label_encoder.classes_
num_labels = len(unique_labels)
print(f"\n라벨 정보:")
print(f"총 라벨 수: {num_labels}")
for i, label in enumerate(unique_labels):
    print(f"  {i}: {label}")

print(f"\nTrain 라벨 분포:")
print(train_df['사용자태그'].value_counts())
print(f"\nTest 라벨 분포:")
print(test_df['사용자태그'].value_counts())

# -------------------------------
# 4. 청크 함수 정의
# -------------------------------

def create_chunked_dataset(df, tokenizer, max_length=512, stride=50):
    """데이터프레임을 청크 데이터셋으로 변환"""
    chunked_rows = []

    for _, row in df.iterrows():
        chunks = chunk_text(row['text'], tokenizer, max_length, stride)

        for chunk in chunks:
            chunk_row = {
                'text': chunk,
                'label': row['label'],
                'patent_id': row.get('출원번호', f'patent_{len(chunked_rows)}')
            }
            chunked_rows.append(chunk_row)

    return pd.DataFrame(chunked_rows)

# -------------------------------
# 4. 청크 함수 및 청크 데이터셋 생성
# -------------------------------
def chunk_text(text, max_length=400, stride=50):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    chunks = []
    start = 0

    while start < len(tokens):
        end = min(start + max_length, len(tokens))
        chunks.append(tokenizer.decode(tokens[start:end], skip_special_tokens=True))
        if end == len(tokens):
            break
        start += max_length - stride
    return chunks

# 원본 데이터를 청크로 분할
train_chunks = []
for idx, row in train_df.iterrows():
    chunks = chunk_text(row['text'])
    for chunk in chunks:
        train_chunks.append({'text': chunk, 'label': row['label'], 'patent_id': idx})

test_chunks = []
for idx, row in test_df.iterrows():
    chunks = chunk_text(row['text'])
    for chunk in chunks:
        test_chunks.append({'text': chunk, 'label': row['label'], 'patent_id': idx})

print(f"청크 전 - Train: {len(train_df)}, Test: {len(test_df)}")
print(f"청크 후 - Train: {len(train_chunks)}, Test: {len(test_chunks)}")

# Dataset 객체 생성 (청크 데이터 사용)
train_dataset_dict = {
    'text': [chunk['text'] for chunk in train_chunks],
    'label': [chunk['label'] for chunk in train_chunks]
}

test_dataset_dict = {
    'text': [chunk['text'] for chunk in test_chunks],
    'label': [chunk['label'] for chunk in test_chunks]
}

# 추론용으로 청크 정보 저장
train_chunk_info = pd.DataFrame(train_chunks)
test_chunk_info = pd.DataFrame(test_chunks)

train_data = Dataset.from_dict(train_dataset_dict)
test_data = Dataset.from_dict(test_dataset_dict)

dataset = DatasetDict({
    'train': train_data,
    'test': test_data
})

print(f"\nDataset 생성 완료:")
print(f"Train: {len(dataset['train'])}개 청크")
print(f"Test: {len(dataset['test'])}개 청크")

Train 데이터 크기: (200, 9)
Test 데이터 크기: (50, 9)
칼럼: ['출원번호', 'DB종류', '특허/실용 구분', '발명의 명칭', '요약', '전체청구항', '대표청구항', '사용자태그', 'WINTELIPS KEY']

합쳐진 텍스트 예시:
발명의 명칭: Method of producing zeolite film | 요약: Provided is a method of producing a zeolite film continuously and efficiently. The method of forming zeolite on a surface of a support is characterized i...

라벨 정보:
총 라벨 수: 5
  0: CPC_C01B
  1: CPC_C01C
  2: CPC_C01D
  3: CPC_C01F
  4: CPC_C01G

Train 라벨 분포:
사용자태그
CPC_C01B    40
CPC_C01C    40
CPC_C01D    40
CPC_C01F    40
CPC_C01G    40
Name: count, dtype: int64

Test 라벨 분포:
사용자태그
CPC_C01B    10
CPC_C01C    10
CPC_C01D    10
CPC_C01F    10
CPC_C01G    10
Name: count, dtype: int64
청크 전 - Train: 200, Test: 50
청크 후 - Train: 903, Test: 207

Dataset 생성 완료:
Train: 903개 청크
Test: 207개 청크


In [5]:
def preprocess_function(examples):
    tokenized = tokenizer(examples['text'], truncation=True, max_length=512)
    tokenized['labels'] = examples['label']
    return tokenized

tokenized_train = dataset['train'].map(preprocess_function, batched=True)
tokenized_test = dataset['test'].map(preprocess_function, batched=True)
tokenized_train = tokenized_train.remove_columns(['text', 'label'])
tokenized_test = tokenized_test.remove_columns(['text', 'label'])

print(f"\n토크나이징 완료:")
print(tokenized_train[0])

Map:   0%|          | 0/903 [00:00<?, ? examples/s]

Map:   0%|          | 0/207 [00:00<?, ? examples/s]


토크나이징 완료:
{'input_ids': [126835, 79632, 20401, 130345, 141676, 25, 6730, 315, 17387, 13703, 337, 632, 4531, 760, 85997, 125535, 25, 53874, 374, 264, 1714, 315, 17387, 264, 13703, 337, 632, 4531, 30878, 323, 29720, 13, 576, 1714, 315, 29064, 13703, 337, 632, 389, 264, 7329, 315, 264, 1824, 374, 31871, 304, 429, 279, 1714, 5646, 25, 264, 1156, 3019, 315, 71808, 13703, 337, 632, 6915, 47373, 311, 264, 7329, 315, 264, 1824, 26, 264, 2086, 3019, 315, 20045, 27268, 17837, 369, 7826, 279, 6915, 47373, 26, 264, 4843, 3019, 315, 10687, 279, 1824, 323, 279, 27268, 17837, 1119, 264, 19259, 37629, 323, 16380, 16643, 68118, 38875, 26, 323, 264, 11737, 3019, 315, 15826, 279, 1824, 389, 892, 13703, 337, 632, 702, 1012, 16643, 696, 76, 745, 91006, 11, 323, 304, 279, 4843, 3019, 11, 279, 9315, 11, 7262, 11, 323, 6396, 315, 279, 27268, 17837, 304, 279, 19259, 37629, 374, 23368, 11, 279, 1824, 374, 7726, 1660, 77208, 304, 279, 27268, 17837, 11, 279, 12720, 882, 315, 279, 16643, 68118, 38875, 374, 23368,

In [6]:
from transformers import DataCollatorWithPadding
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.metrics import classification_report
import torch.nn.functional as F
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
def compute_metrics(pred):
    labels=pred.label_ids
    preds=pred.predictions.argmax(-1)
    precision, recall, f1,_ =precision_recall_fscore_support(labels, preds, average='macro')
    acc=accuracy_score(labels,preds)
    print("\n Classification Report")
    print(classification_report(labels, preds, digits=2))
    logits_tensor=torch.tensor(pred.predictions)
    labels_tensor=torch.tensor(pred.label_ids)
    loss=F.cross_entropy(logits_tensor,labels_tensor).item()
    return{
        'accuracy':acc,
        'f1':f1,
        'precision':precision,
        'recall':recall,
        'eval_loss':loss
    }

In [7]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
peft_config=LoraConfig(
    lora_alpha=128,
    lora_dropout=0.1,
    r=64,
    bias='none',
    task_type='SEQ_CLS',
    target_modules=['k_proj','gate_proj','v_proj','up_proj','q_proj','o_proj','down_proj']
)
model=prepare_model_for_kbit_training(model)
model=get_peft_model(model,peft_config)

In [10]:
from trl import SFTConfig
output_dir= "../output"
training_arguments=SFTConfig(
    output_dir=output_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    optim='paged_adamw_32bit',
    lr_scheduler_type='cosine',
    num_train_epochs=2,
    warmup_steps=50,
    logging_steps=10,
    fp16=True,
    gradient_checkpointing=True,
    dataset_text_field='text',
    max_length=512,
    label_names=['labels']
)

In [11]:
import os
from trl import SFTTrainer
trainer=SFTTrainer(
    model=model,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    processing_class=tokenizer,
    args=training_arguments,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    peft_config=peft_config
)
trainer.train()
print(trainer.evaluate())
trainer.model.save_pretrained('Qwen3-0.6B-QLoRA2')

Truncating train dataset:   0%|          | 0/903 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/207 [00:00<?, ? examples/s]

Step,Training Loss
10,5.100200
20,3.832300
30,4.069600
40,4.433300
50,4.259000
60,4.082900
70,3.298300
80,3.786300
90,3.478700
100,3.783600



 Classification Report
              precision    recall  f1-score   support

           0       0.47      0.48      0.48        50
           1       0.73      0.73      0.73        37
           2       0.82      0.62      0.71        37
           3       0.52      0.82      0.64        45
           4       0.55      0.29      0.38        38

    accuracy                           0.59       207
   macro avg       0.62      0.59      0.59       207
weighted avg       0.61      0.59      0.58       207

{'eval_loss': 1.1379822492599487, 'eval_accuracy': 0.5893719806763285, 'eval_f1': 0.5859821882969716, 'eval_precision': 0.6185746594031597, 'eval_recall': 0.58860945155682, 'eval_runtime': 188.3527, 'eval_samples_per_second': 1.099, 'eval_steps_per_second': 0.552}


In [12]:
from transformers import AutoModelForSequenceClassification
from peft import PeftModel

# -------------------------------
# QLoRA SEQ_CLS 어댑터 머지
# -------------------------------

print("=== QLoRA Adapter Merge ===")

# 1. 베이스 모델을 SEQ_CLS로 직접 로드 (어댑터 훈련 시와 동일)
base_model = AutoModelForSequenceClassification.from_pretrained(
    "Qwen/Qwen3-0.6B",
    num_labels=5,
    ignore_mismatched_sizes=False
)

print("Base model structure:")
for name, _ in base_model.named_modules():
    if 'score' in name or 'classifier' in name:
        print(f"  Found: {name}")

# 2. 어댑터 로드 및 머지
adapter_path = "/Qwen3-0.6B-QLoRA"

model = PeftModel.from_pretrained(
    base_model,
    adapter_path,
    ignore_mismatched_sizes=False,
    device_map='auto'
)

# 3. 머지 및 저장
merged_model = model.merge_and_unload()
merged_model.save_pretrained("C:/wips/output/Qwen3_merged_method3")
print("머지 완료!")

=== QLoRA Adapter Merge ===


Some weights of Qwen3ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen3-0.6B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Base model structure:
  Found: score
머지 완료!


In [1]:
print("머지 완료!")
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score
from datasets import load_from_disk, DatasetDict
import numpy as np

# 데이터 로드 (기존 코드 그대로)
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.preprocessing import LabelEncoder
import numpy as np

# -------------------------------
# 1. 엑셀 데이터 로드 및 전처리
# -------------------------------

# 엑셀 파일 경로
train_excel_path = "C:/Users/nice.DESKTOP-0JCR5PF/Desktop/train.xlsx"  # 200행
test_excel_path = "C:/Users/nice.DESKTOP-0JCR5PF/Desktop/test.xlsx"  # 50행

# 엑셀 읽기
train_df = pd.read_excel(train_excel_path)
test_df = pd.read_excel(test_excel_path)

# 텍스트 칼럼들을 합치기
text_columns = ['발명의 명칭', '요약', '전체청구항', '대표청구항']


def combine_text_columns(row):
    texts = []
    for col in text_columns:
        if pd.notna(row[col]) and str(row[col]).strip():
            texts.append(f"{col}: {str(row[col]).strip()}")
    return " | ".join(texts)


train_df['text'] = train_df.apply(combine_text_columns, axis=1)
test_df['text'] = test_df.apply(combine_text_columns, axis=1)

# 라벨 인코딩
label_encoder = LabelEncoder()
all_labels = pd.concat([train_df['사용자태그'], test_df['사용자태그']])
label_encoder.fit(all_labels.dropna())

train_df['label'] = label_encoder.transform(train_df['사용자태그'])
test_df['label'] = label_encoder.transform(test_df['사용자태그'])

# Dataset 객체로 변환
train_data = Dataset.from_dict({
    'text': train_df['text'].tolist(),
    'label': train_df['label'].tolist()
})

test_data = Dataset.from_dict({
    'text': test_df['text'].tolist(),
    'label': test_df['label'].tolist()
})

# -------------------------------
# 2. 기존 방식과 동일한 샘플링
# -------------------------------

# 기존 코드와 동일한 구조
ds = DatasetDict({'train': train_data, 'test': test_data})
train = ds["train"]
test = ds["test"]

num_labels = len(set(train["label"]))
train_per_label = 200 // num_labels  # 전체 200개를 라벨별로 균등 분배
test_per_label = 50 // num_labels  # 전체 50개를 라벨별로 균등 분배

print(f"라벨 수: {num_labels}")
print(f"라벨별 train 샘플: {train_per_label}개")
print(f"라벨별 test 샘플: {test_per_label}개")


def sample_per_label(dataset, per_label, seed=42):
    labels = np.array(dataset["label"])
    picked = []
    rng = np.random.default_rng(seed)
    for l in range(num_labels):
        idx = np.where(labels == l)[0]
        if len(idx) < per_label:
            print(f"경고: 라벨 {l}에 {len(idx)}개 샘플만 있음 (필요: {per_label}개)")
            picked.extend(idx)  # 있는 만큼만 추가
        else:
            picked.extend(rng.choice(idx, per_label, replace=False))
    return dataset.select(sorted(picked))


train_small = sample_per_label(train, train_per_label)
test_small = sample_per_label(test, test_per_label)

# 라벨 분포 확인
from collections import Counter

print(f"\nTrain 라벨 분포: {Counter(train_small['label'])}")
print(f"Test 라벨 분포: {Counter(test_small['label'])}")

# 원본 라벨로 변환해서 확인
train_original_labels = [label_encoder.inverse_transform([label])[0] for label in train_small['label']]
test_original_labels = [label_encoder.inverse_transform([label])[0] for label in test_small['label']]

print(f"\nTrain 원본 라벨 분포: {Counter(train_original_labels)}")
print(f"Test 원본 라벨 분포: {Counter(test_original_labels)}")

# CUDA 및 디바이스 확인
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"현재 GPU 메모리 사용량: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")

# 토크나이저 로드
print("토크나이저 로딩 중...")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("토크나이저 로딩 완료!")

# 모델 로드 (수정된 부분)
print("모델 로딩 중...")
model = AutoModelForSequenceClassification.from_pretrained(
    r"/output/Qwen3_merged_method2",
    num_labels=num_labels,  # 동적으로 계산된 라벨 수 사용
    torch_dtype=torch.float16,
    device_map="auto",  # 자동 디바이스 배치만 사용
    low_cpu_mem_usage=True
)
print("모델 로딩 완료!")

# pad_token_id 설정
model.config.pad_token_id = tokenizer.pad_token_id

# 모델 정보 출력
print(f"모델 디바이스: {next(model.parameters()).device}")
print(f"모델 dtype: {next(model.parameters()).dtype}")

# GPU 메모리 사용량 확인
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"GPU 메모리 사용량: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")


# -------------------------------
# 3. 청크 함수 정의 및 청크 생성
# -------------------------------

def chunk_text(text, tokenizer, max_length=400, stride=50):
    """텍스트를 청크로 분할하는 함수"""
    tokens = tokenizer.encode(text, add_special_tokens=False)
    chunks = []
    start = 0

    while start < len(tokens):
        end = min(start + max_length, len(tokens))
        chunks.append(tokenizer.decode(tokens[start:end], skip_special_tokens=True))
        if end == len(tokens):
            break
        start += max_length - stride

    return chunks


print("\n청크 데이터 생성 중...")

# Train 데이터 청크 생성
train_chunks = []
train_patent_info = []
for idx in range(len(train_small)):
    text = train_small[idx]['text']
    label = train_small[idx]['label']
    chunks = chunk_text(text, tokenizer)

    # 각 청크를 저장
    for chunk in chunks:
        train_chunks.append({
            'text': chunk,
            'label': label,
            'patent_id': idx,
            'chunk_len': len(chunk.split())
        })

    # 특허별 정보 저장
    train_patent_info.append({
        'patent_id': idx,
        'original_text': text,
        'label': label,
        'chunk_count': len(chunks)
    })

# Test 데이터 청크 생성
test_chunks = []
test_patent_info = []
for idx in range(len(test_small)):
    text = test_small[idx]['text']
    label = test_small[idx]['label']
    chunks = chunk_text(text, tokenizer)

    # 각 청크를 저장 (patent_id는 train 이후 번호로)
    for chunk in chunks:
        test_chunks.append({
            'text': chunk,
            'label': label,
            'patent_id': idx + len(train_small),  # train 이후 번호
            'chunk_len': len(chunk.split())
        })

    # 특허별 정보 저장
    test_patent_info.append({
        'patent_id': idx + len(train_small),
        'original_text': text,
        'label': label,
        'chunk_count': len(chunks)
    })

print(f"Train: {len(train_small)}개 특허 → {len(train_chunks)}개 청크")
print(f"Test: {len(test_small)}개 특허 → {len(test_chunks)}개 청크")

# 전체 청크 데이터
all_chunks = train_chunks + test_chunks
all_chunk_texts = [chunk['text'] for chunk in all_chunks]

print(f"전체 청크 수: {len(all_chunks)}개")

# -------------------------------
# 4. 청크별 예측 수행
# -------------------------------

model.eval()
chunk_predictions = []

print("\n청크별 예측 수행 중...")
batch_size = 8  # 배치 사이즈를 변수로 분리

with torch.no_grad():
    for i in range(0, len(all_chunk_texts), batch_size):
        batch_texts = all_chunk_texts[i:i + batch_size]

        # 토크나이저 처리 시 디바이스 확인
        inputs = tokenizer(batch_texts, truncation=True, padding=True,
                           max_length=512, return_tensors="pt")

        # 입력을 모델과 같은 디바이스로 이동
        device = next(model.parameters()).device
        inputs = {k: v.to(device) for k, v in inputs.items()}

        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1)
        chunk_predictions.extend(probs.cpu().numpy())

        # 진행 상황 출력
        if (i // batch_size + 1) % 50 == 0:
            print(f"진행 중: {i + len(batch_texts)}/{len(all_chunk_texts)} 청크 완료")

print(f"총 {len(chunk_predictions)}개 청크 예측 완료")


# -------------------------------
# 5. 특허별 예측 집계 (가중 평균)
# -------------------------------

def aggregate_patent_predictions(chunks, chunk_preds, start_idx=0):
    """특허별로 청크 예측을 가중 평균으로 집계"""
    patent_preds = []
    patent_labels = []

    # 특허별로 그룹화
    patent_groups = {}
    for i, chunk in enumerate(chunks):
        patent_id = chunk['patent_id']
        if patent_id not in patent_groups:
            patent_groups[patent_id] = []
        patent_groups[patent_id].append((i + start_idx, chunk))

    # 각 특허별로 가중 평균 계산
    for patent_id in sorted(patent_groups.keys()):
        chunk_indices_and_info = patent_groups[patent_id]

        # 해당 특허의 청크 예측값들과 가중치 수집
        patent_chunk_preds = []
        weights = []

        for chunk_idx, chunk_info in chunk_indices_and_info:
            patent_chunk_preds.append(chunk_preds[chunk_idx])
            weights.append(chunk_info['chunk_len'])  # 청크 길이를 가중치로 사용

        # 가중 평균 계산
        patent_chunk_preds = np.array(patent_chunk_preds)
        weights = np.array(weights)

        weighted_pred = np.average(patent_chunk_preds, axis=0, weights=weights)
        final_pred = np.argmax(weighted_pred)

        patent_preds.append(final_pred)
        patent_labels.append(chunk_indices_and_info[0][1]['label'])  # 모든 청크가 같은 라벨

    return patent_preds, patent_labels


# Train과 Test 각각 집계
print("\n특허별 예측 집계 중...")
train_preds, train_labels = aggregate_patent_predictions(
    train_chunks, chunk_predictions, start_idx=0
)

test_preds, test_labels = aggregate_patent_predictions(
    test_chunks, chunk_predictions, start_idx=len(train_chunks)
)

# -------------------------------
# 6. 결과 출력
# -------------------------------

# 전체 결과
all_preds = train_preds + test_preds
all_true_labels = train_labels + test_labels

total_accuracy = accuracy_score(all_true_labels, all_preds)
train_accuracy = accuracy_score(train_labels, train_preds)
test_accuracy = accuracy_score(test_labels, test_preds)

print(f"\n=== 청크 기반 예측 결과 ===")
print(f"전체 정확도: {total_accuracy:.4f} ({total_accuracy * 100:.2f}%)")
print(f"학습 데이터 정확도: {train_accuracy:.4f} ({train_accuracy * 100:.2f}%)")
print(f"검증 데이터 정확도: {test_accuracy:.4f} ({test_accuracy * 100:.2f}%)")

print(f"\n상세 통계:")
print(f"- 전체 특허: {len(all_true_labels)}개")
print(f"- 전체 청크: {len(chunk_predictions)}개")
print(f"- 평균 청크/특허: {len(chunk_predictions) / len(all_true_labels):.1f}개")

# 라벨별 정확도도 확인
from sklearn.metrics import classification_report

print(f"\n=== 상세 분류 리포트 ===")
target_names = [label_encoder.inverse_transform([i])[0] for i in range(num_labels)]
print(classification_report(all_true_labels, all_preds, target_names=target_names, digits=4))

머지 완료!
라벨 수: 5
라벨별 train 샘플: 40개
라벨별 test 샘플: 10개

Train 라벨 분포: Counter({0: 40, 1: 40, 2: 40, 3: 40, 4: 40})
Test 라벨 분포: Counter({0: 10, 1: 10, 2: 10, 3: 10, 4: 10})

Train 원본 라벨 분포: Counter({'CPC_C01B': 40, 'CPC_C01C': 40, 'CPC_C01D': 40, 'CPC_C01F': 40, 'CPC_C01G': 40})
Test 원본 라벨 분포: Counter({'CPC_C01B': 10, 'CPC_C01C': 10, 'CPC_C01D': 10, 'CPC_C01F': 10, 'CPC_C01G': 10})
CUDA 사용 가능: True
GPU 이름: NVIDIA GeForce GTX 1660 Ti
GPU 메모리: 6.00 GB
현재 GPU 메모리 사용량: 0.00 GB
토크나이저 로딩 중...


`torch_dtype` is deprecated! Use `dtype` instead!


토크나이저 로딩 완료!
모델 로딩 중...
모델 로딩 완료!
모델 디바이스: cuda:0
모델 dtype: torch.float16
GPU 메모리 사용량: 1.11 GB

청크 데이터 생성 중...
Train: 200개 특허 → 903개 청크
Test: 50개 특허 → 207개 청크
전체 청크 수: 1110개

청크별 예측 수행 중...
진행 중: 400/1110 청크 완료
진행 중: 800/1110 청크 완료
총 1110개 청크 예측 완료

특허별 예측 집계 중...

=== 청크 기반 예측 결과 ===
전체 정확도: 0.4400 (44.00%)
학습 데이터 정확도: 0.4650 (46.50%)
검증 데이터 정확도: 0.3400 (34.00%)

상세 통계:
- 전체 특허: 250개
- 전체 청크: 1110개
- 평균 청크/특허: 4.4개

=== 상세 분류 리포트 ===
              precision    recall  f1-score   support

    CPC_C01B     0.2963    0.1600    0.2078        50
    CPC_C01C     0.6667    0.7200    0.6923        50
    CPC_C01D     0.3404    0.6400    0.4444        50
    CPC_C01F     0.4444    0.4800    0.4615        50
    CPC_C01G     0.4762    0.2000    0.2817        50

    accuracy                         0.4400       250
   macro avg     0.4448    0.4400    0.4176       250
weighted avg     0.4448    0.4400    0.4176       250

